# Что бывает после предлогов, управляющих аблативом (кроме аблатива)

### Библиотека cltk

In [ ]:
from cltk.corpus.utils.importer import CorpusImporter
from cltk.corpus.readers import get_corpus_reader
from cltk.tag.pos import POSTag
from cltk.lemmatize.latin.backoff import BackoffLatinLemmatizer
from cltk.tokenize.latin.sentence import SentenceTokenizer
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
from datetime import datetime

In [ ]:
import random
import json
from datetime import datetime
from itertools import zip_longest, islice

### Графики

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 
import matplotlib.pyplot as plt
import seaborn as sns

Библиотеки для отслеживания памяти и измерения времени, в итоговом билде они не нужны

In [ ]:
# import cProfile
# import pstats
# from pstats import SortKey
# import sys
# import os
# os.environ["PYTHONTRACEMALLOC"] = '10'
# import tracemalloc

In [ ]:
# tracemalloc.start()

In [ ]:
pd.set_option("display.max_rows", 40)

### Корпус perseus из cltk и модель

In [ ]:
corpus_importer = CorpusImporter('latin')

corpus_importer.import_corpus('latin_models_cltk')
corpus_importer.import_corpus('latin_text_perseus')

reader = get_corpus_reader(language='latin', corpus_name='latin_text_perseus')

In [ ]:
tagger = POSTag('latin')
lemmatizer = BackoffLatinLemmatizer()
sent_tokenizer = SentenceTokenizer(strict=True)

## Основная часть проекта

### Соответствие кратких тегов нормальным словам

In [ ]:
# def get_categories():
pos = {
    'n': 'noun',
    'v': 'verb',
    't': 'participle',
    'a': 'adjective',
    'd': 'adverb',
    'c': 'conjunction',
    'r': 'preposition',
    'p': 'pronoun',
    'm': 'numeral',
    'i': 'interjection',
    'e': 'exclamation',
    'u': 'punctuation'
}

person = {
    '1': 'first person',
    '2': 'second person',
    '3': 'third person'
}

number = {
    's': 'singular',
    'p': 'plural'
}

tense = {
    'p': 'present',
    'i': 'imperfect',
    'r': 'perfect',
    'l': 'pluperfect',
    't': 'future perfect',
    'f': 'future',
}

mood = {
    'i': 'indicative',
    's': 'subjunctive',
    'n': 'infinitive',
    'm': 'imperative',
    'p': 'participle',
    'd': 'gerund',
    'g': 'gerundive',
    'u': 'supine',
}

voice = {
    'a': 'active',
    'p': 'passive',
}

gender = {
    'm': 'masculine',
    'f': 'feminine',
    'n': 'neuter',
}

case = {
    'n': 'nominative',
    'g': 'genitive',
    'd': 'dative',
    'a': 'accusative',
    'b': 'ablative',
    'v': 'vocative',
    'l': 'locative',
}

degree = {
    'c': 'comparative',
    's': 'superlative',
}

categories = {'pos': pos, 'person': person, 'number': number, 'tense': tense,
              'mood': mood, 'voice': voice, 'gender': gender, 'case': case,
              'degree': degree}
categories_names = {1: 'pos', 2: 'person', 3: 'number', 4: 'tense',
                    5: 'mood', 6: 'voice', 7: 'gender', 8: 'case',
                    9: 'degree'}
    
#     return categories, categories_names

Функция ```convert_analysis_to_dict()``` конвертирует 9 символьную строку анализа, в словарь "категория - значение".

In [ ]:
def convert_analysis_to_dict(analysis, keep_empty=True):   
    dict_analysis = {}
    
    for i, cat_value_letter in enumerate(analysis, start=1):
        cat_name = categories_names[i]
        if cat_value_letter == '-':
            if keep_empty:
                dict_analysis[cat_name] = 'N/A'
            continue
        try:    
            cat_value_word = categories[cat_name][cat_value_letter.lower()]
            dict_analysis[cat_name] = cat_value_word
        except KeyError as k_e:
            print(k_e)
            print(cat_value_letter, analysis)
            
    return dict_analysis

Функция ```is_foreign_lang_in_sent()``` возвращает ```False```, если в предложении есть хотя бы один странный символ. Они используются в корпусе для кодирования иностранных языков внутри текста.

In [ ]:
def is_foreign_lang_in_sent(sent):
    # истинно для открывков вроде
    # h(\ to/sa fa/rmaka h)/dh, o(/sa tre/fei eu)rei=a xqw/n
    foreign_chars = set(['\\', '/', '|', '='])
    return any(word in sent for word in foreign_chars)

Функция ```get_non_ablative_after_preposition()``` находит в предложении предлоги, употребляющиеся с аблативом, и стоящие после них слова. Если есть что-то необычное, например, слово не в аблативе, или не имеющее падежа, то оно возвращается.

Сперва сделаем разборы слов предложения, заодно они токенизируются. Все слова на время сохраним в `words_in_sent`. Если среди них нет ни одного нужного предлога, функция сразу вернёт `None`. Если предлоги есть, то идёт проход по всем словам, проверка каждого слова и если это нужный предлог, то проверяется следующее слово. Если предлог - последнее слово в предложении (вернее в клаузе), то результат обозначается специальным маркером `%END%`, в противном случае анализируется следующее слово.

In [ ]:
def get_non_ablative_after_preposition(sent, context_size=8):   
    
    abl_preps = set(('a', 'ab', 'de', 'cum', 'ex', 'e', 'sine', 'pro', 'prae'))
    results = {'sentence': None, 'strange_pairs': [], 'contexts': []}
    
    tagged_words =  tagger.tag_ngram_123_backoff(sent)
    words_number = len(tagged_words)
    
    words_in_sent, _ = zip(*tagged_words)
    if not any(prep in words_in_sent for prep in abl_preps):
        return None
    
    for i, (word, analysis) in enumerate(tagged_words):        
        if analysis is None:
            continue
        
        is_needed_prep = analysis[0] == 'R' and word in abl_preps
        if not is_needed_prep:
            continue
        
        verbose_analysis = convert_analysis_to_dict(analysis)
        verbose_analysis['word'] = word
        [(_, word_lemma)] = lemmatizer.lemmatize([word])
        verbose_analysis['lemma'] = word_lemma

        is_last = i == words_number - 1        
        if is_last:
            results['sentence'] = sent
            results['strange_pairs'].append(
                (verbose_analysis,
                 {cat_name: '%END%' for cat_name in list(categories.keys())+['word', 'lemma']}))
            results['contexts'].append(words_in_sent[max(i-context_size, 0):])
            
        else:
            next_word, next_word_analysis = tagged_words[i+1]
            if next_word_analysis is None:
                # может всё-таки брать такие слова?
                continue
                
            if next_word_analysis[8-1].lower() != 'b':
                next_word_verbose_analysis = convert_analysis_to_dict(
                    next_word_analysis)
                next_word_verbose_analysis['word'] = next_word
                [(_, next_word_lemma)] = lemmatizer.lemmatize([next_word])
                next_word_verbose_analysis['lemma'] = next_word_lemma
                
                results['sentence'] = sent
                results['strange_pairs'].append((verbose_analysis,
                                                next_word_verbose_analysis))
                results['contexts'].append(words_in_sent[max(i-context_size, 0):min(i+context_size, words_number)])
            
    if results['sentence'] is None:
        return None
    return results

Функция для обработки одного документа. Достаёт весь текст, соединённый в строку. (Точнее это генератор, возвращающий по одном куску текста).

In [ ]:
def flatten_dict_into_str(iter_):
    if isinstance(iter_, str):
        yield iter_
    else:
        try:
            for i, obj in iter_.items():
                yield from flatten_dict_into_str(obj)
        except:
            try:
                for obj in iter_:
                    yield from flatten_dict_into_str(obj)
            except:
                yield iter_

Функция `grouper()` из [itertools recipes](https://docs.python.org/library/itertools.html#itertools-recipes)

> this is feeding the same iterator to `izip_longest` multiple times, causing it to consume successive values of the same sequence rather than striped values from separate sequences. [StackOverflow](https://stackoverflow.com/questions/434287/what-is-the-most-pythonic-way-to-iterate-over-a-list-in-chunks)

Ещё вариант:

```python
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, > len(seq), size))
```



In [ ]:
def grouper(iterable, chunk_size, fillvalue=None):
    args = [iter(iterable)] * chunk_size
    return zip_longest(*args, fillvalue=fillvalue)

Функция ```analyse_document()``` склеивает все предложения в документе в единый текст, а потом разбивает его по предложениям, анализируя предложения группами, в зависимости от их количества в документе. Если "предложение" это ```None``` или в нём есть вкрапления на другом языке, то предложение пропускается. Если же всё хорошо, то к предложению применяется описанная выше ```get_non_ablative_after_preposition()```. Если всё совсем хорошо и функция вернула непустое значение, то оно записывается в словарь данных по документу.

In [ ]:
def analyse_document(doc):
    analysis_for_doc = {'author': doc['author'], 'title': doc['originalTitle'],
                       'sentences': []}
    
    text = ' '.join([str(el) for el in flatten_dict_into_str(doc['text'])])
    sentences = sent_tokenizer.tokenize(text)
    
    ### chunking
    num_of_sent = len(sentences)
    chunk_size = 300
    if num_of_sent >= 2 * chunk_size:
        sentences = grouper(sentences, chunk_size)
    else:
        sentences = [sentences]
    
    tqdm.write('Now analyzing ' + doc['author'] + ' ' + doc['originalTitle'])
    tqdm.write('Num of sentences: ' + str(num_of_sent))
    tqdm.write(str(type(sentences)))
    
    for sent_chunk in tqdm(sentences):
        for sent in sent_chunk:
            # если в предложении вкрапления греческого, которые в корпусе 
            # записаны очень странно, то пропускаем это предложение
            # также пропускаем пустое предложение, которое могло появиться из-за
            # fillvalue в итераторе 
            if sent is None or is_foreign_lang_in_sent(sent):
                continue

            sent_data = get_non_ablative_after_preposition(sent)
            if sent_data is None:
                continue

            analysis_for_doc['sentences'].append(sent_data)
    
    if not analysis_for_doc['sentences']:
        return None
    
    return analysis_for_doc

Функция ```get_data_from_x_docs()``` возвращает все данные для нескольких документов.

In [ ]:
def get_data_from_x_docs(docs, count_doc, start=0):
    results = []

    for doc in tqdm(islice(docs, start, count_doc)):
        result = analyse_document(doc)
        if result is not None:
            results.append(result)
            
    return results

В качестве `docs` взят генератор. Это довольно быстро по времени, и экономнее по памяти. **NB**: характерные гребни на графике использования RAM когда выполняется цикл, видимо всё-таки из-за кода в функции выше:

```python
text = ' '.join([str(el) for el in flatten_dict_into_str(doc['text'])])
sentences = sent_tokenizer.tokenize(text)
```

In [ ]:
docs = reader.docs()

Берём сколько надо документов. Здесь данные, чтобы взять весь корпус (293 документа).

In [ ]:
count_doc = 293
results = get_data_from_x_docs(docs, count_doc)
time = datetime.now.strftime('%H.%M.%S-%d-%m')
with open('out_x{}-{}.json'.format(count_doc, time), 'w', encoding='utf-8') as f:
    json.dump(docs, f, ensure_ascii=False, indent=4)
# end = time.time()

In [ ]:
time = datetime.now().strftime('%H.%M.%S-%d-%m')
with open('out_x{}-{}.json'.format(count_doc, time), 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

## Импорт из json

Все данные могут быть сериализованы в json, т.к. данные - список словарей и внутри всё списки/словари. Выше мы сохранили данные в json, а здесь может их считать при последующем запуске программы.

In [ ]:
with open('out_x293-19.05.58-25-12.json', 'r', encoding='utf-8') as f:
    results = json.load(f)

Сделать из списка словарей, каждый из которых описывает один документ, датафрейм. Каждой паре слов из словаря присвоить метаданные

In [ ]:
def convert_dict_to_dataframe(results):
    ## пока не учитываем ['contexts'], собравшиеся неправильно
    
    author = results['author']
    title = results['title']
    entries_list = []
    
    for sentence_and_word_pairs_dict in results['sentences']:
        sentence = sentence_and_word_pairs_dict['sentence'].replace('\n', ' ')
        word_pairs = sentence_and_word_pairs_dict['strange_pairs']
        contexts_for_pairs = sentence_and_word_pairs_dict['contexts']
        for prep_dict, second_word_dict in word_pairs:
            entry = {'prep': prep_dict['word'], **second_word_dict,
                     'sentence': sentence,
                     'author': author, 'title': title}
            entries_list.append(entry)
    
    
    df = pd.DataFrame(entries_list)
    cols = ['prep', 'word', 'lemma', 'pos', 'person', 'number', 'tense', 'mood',
            'voice', 'gender', 'case', 'degree', 'sentence', 'author', 'title']
    df = df[cols]  
    
    df.replace('N/A', np.nan, inplace=True)
    
    dtypes = {cat_name: CategoricalDtype(categories=cat_map.values())
              for cat_name, cat_map in categories.items()}
    dtypes['prep'] = 'category'
    
    df_new = df.astype(dtypes)
    
    return df_new

In [ ]:
df_full = pd.concat(list(map(convert_dict_to_dataframe, results)))

In [146]:
df_full.describe()

,prep,word,lemma,pos,person,number,tense,mood,voice,gender,case,degree,sentence,author,title
count,12460,12460,12460,12129,578,9253,781,779,781,8665,8661,26,12460,12460,12460
unique,9,1556,1008,11,3,2,6,7,2,3,6,2,11617,43,270
top,a,me,punc,pronoun,third person,singular,present,indicative,active,masculine,accusative,superlative,pro di immortales!,cicero,Naturalis Historia
freq,2612,937,1244,3662,304,6915,448,238,612,4513,3412,13,8,4125,2341


In [ ]:
df_full[(pd.isnull(df_full['pos']))].groupby('word',as_index=False).size()#[['prep', 'word', 'lemma', 'pos', 'sentence']].head())

In [ ]:
df_x = df_full[pd.notnull(df_full['pos'])]

In [147]:
df_x.describe()

,prep,word,lemma,pos,person,number,tense,mood,voice,gender,case,degree,sentence,author,title
count,12129,12129,12129,12129,578,9253,781,779,781,8665,8661,26,12129,12129,12129
unique,9,1543,999,11,3,2,6,7,2,3,6,2,11339,43,270
top,a,me,punc,pronoun,third person,singular,present,indicative,active,masculine,accusative,superlative,pro di immortales!,cicero,Naturalis Historia
freq,2514,937,1244,3662,304,6915,448,238,612,4513,3412,13,8,4111,2062


In [ ]:
df_full[['pos', 'word', 'lemma', 'sentence']].groupby(['pos', 'word'])['word'].count()

In [ ]:
for pos in pos_general['pos']:
    example_words = df_x[df_x['pos'] == pos].groupby('word')['lemma'].count()
    print('pos is', pos, '\n', 
          example_words, '\n')

In [ ]:
df_x[['prep', 'pos', 'word']].groupby(['prep', 'pos'])['word'].count()

In [ ]:
df_x[df_x['word'] == '%END%']

In [ ]:
df_x[df_x['pos'] == 'preposition'][['prep','word', 'sentence']].groupby(
    ['prep', 'word', 'sentence']
)['word'].count()

# Сделать статистику для:

* Часть речи
    > Предлог -> часть речи и наоборот
    
* Для частей речи склоняющихся по падежу:
    > Предлог -> Падеж (наоборот тоже?)
    
    Все вместе, и по каждой в отдельности
    
    
**ЕСТЬ**:
* Части речи
* Предлог -> падеж (в целом)
* Часть речи -> падеж (для каждой части речи)
* Несклоняемая часть речи -> x частых лемм
* Автор

## Статистика по собранным данным

```len(results)``` -  276 (а всего-то 293!)

### Количество по части речи

In [ ]:
pos_general = df_x.groupby(['pos'], as_index=False)['word'].count()
pos_general_sorted = pos_general.sort_values(['word'], ascending=False).reset_index(drop=True)
pos_general_sorted_nonull = pos_general_sorted[pos_general_sorted['word'] != 0]
pos_general

In [ ]:
f, ax = plt.subplots(figsize=(18, 14))
sns_plot = sns.barplot(x='pos', y='word', data=pos_general_sorted_nonull, order=pos_general_sorted_nonull['pos'].to_list())
# sns_plot = sns.countplot(x='pos', data=df_x) - то же самое
sns_plot.set(yscale="log")
plt.ylabel('Count')
for bar in sns_plot.patches:
        height = bar.get_height()
        if not np.isnan(height):
            height = int(height)
        if height == 0:
            continue

        ax.text(bar.get_x() + bar.get_width() / 2, height + 5, f'{height}',
                ha='center', va='bottom')
        
print(sns_plot)
plt.savefig('graphics/boxplot_posCount.png')

### Количество по авторам

In [ ]:
def count_by_author(author='all', most_common=5, plot_what='Count'):
    df = df_x
    total = len(df[pd.notnull(df['author'])])

    if author != 'all':
        df = df_x[df_x['author'] == 'author']
        
    author_general = df.groupby('author', as_index=False).size()
    author_general_sorted = author_general.sort_values(ascending=False).reset_index()
    author_ratio = author_general_sorted.assign(ratio=lambda x: x[0] / total)
    author_ratio.astype({0: int})
    
    print('\n', author_ratio)
    print(author_general_sorted.index, author_general_sorted.columns)
    
    f, ax = plt.subplots(figsize=(18, 14))
    if plot_what == 'Count':
        author_plot = sns.barplot(x='author', y=0, data=author_ratio.loc[0:most_common])
    elif plot_what == 'Ratio':
        author_ratio = author_ratio.sort_values('ratio', ascending=False)
        author_plot = sns.barplot(x='author', y='ratio', data=author_ratio.loc[0:most_common])
    else:
        raise('wrong input')
    
    plt.xticks(rotation=25)
    plt.ylabel(f'{plot_what}')
    plt.title(f'{plot_what} of entries by author')
    
    for bar in author_plot.patches:
            height = bar.get_height()
            if not np.isnan(height) and not height is int:
                height = round(float(height), 2)
            elif height is int:
                height = int(height)
            if height == 0:
                continue

            ax.annotate(f'{height}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points", ha='center', va='bottom')

    print(author_plot)
    plt.savefig(f'graphics/boxplot_author{plot_what}.png')

In [ ]:
count_by_author()

### Количество по падежу по части речи

In [ ]:
def groupby_and_plot_by_pars(filter_col, filter_col_values, group_on, aggr_on, df=df_x, plot_null=False, want_plot=True):
    if not filter_col or not filter_col_values:
        df_fragment = df
    else:
        df_fragment = df[df[filter_col] == filter_col_values]
        
    grouped_df_for_col = df_fragment.groupby(group_on, as_index=False)[aggr_on].count()
    
    sorted_groups = grouped_df_for_col.sort_values([aggr_on], ascending=False).reset_index(drop=True)
    if not plot_null:
        sorted_groups = sorted_groups[sorted_groups[aggr_on] != 0]
    print(f'Groupping df_x where "{filter_col}" only has values: "{filter_col_values}". '
          f'Groupping by "{group_on}". (aggregation by "{aggr_on}")\n\n', sorted_groups, '\n')
    
    if not want_plot:
        return
    
    ### сохранять цвет
#     if group_on == 'case':
#         colors = {'nominative': 'C0'
#                     'genitive': 'C1'
#                     'dative': 'C2'
#                     'accusative': 'C3'
#                     'vocative': 'C4'
#                     'locative': 'C5'}

    
    f, ax = plt.subplots(figsize=(18, 14))
    groupon_values = sns.barplot(x=group_on, y=aggr_on, 
                                 data=sorted_groups, order=sorted_groups[group_on].to_list())
    groupon_values.set(yscale="log")
    plt.ylabel('Count')
    plt.title(f'Number of {filter_col_values}s by {group_on}')
    
    for bar in groupon_values.patches:
            height = bar.get_height()
            if not np.isnan(height):
                height = int(height)
            if height == 0:
                continue

            ax.annotate(f'{height}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points", ha='center', va='bottom')

    print(groupon_values)
    plt.savefig(f'graphics/boxplot_{filter_col}={filter_col_values}_{group_on}.png')

In [ ]:
pos_with_case = ['pronoun', 'noun', 'adjective', 'participle']
for pos in pos_with_case:
    groupby_and_plot_by_pars('pos', pos, 'case', 'word', plot_null=False)

[Настаканный график](https://stackoverflow.com/questions/26683654/making-a-stacked-barchart-in-pandas)

### Количество падежей по предлогу

In [ ]:
def case_by_prep(prep='all'):
    
    df = df_x
    if not prep == 'all':
        df = df_x[df_x['prep'].isin(prep)]
    
    df_for_case = df[pd.notnull(df['case'])][['prep', 'case', 'word']]

    case_by_prep_groupby = df_for_case.groupby(['prep', 'case'], as_index=False).size()
    sorted_case_by_prep_groupby = case_by_prep_groupby.sort_values(ascending=False)
    df_case_by_prep = sorted_case_by_prep_groupby.astype('int').unstack()
    df_case_by_prep

    sum_by_prep = df_case_by_prep.sum(axis=1).reset_index()
    sorted_sum_by_prep = sum_by_prep.sort_values(0, ascending=False)

    ordered_case_by_prep = df_case_by_prep.reindex(sorted_sum_by_prep['prep'].to_list())
    print(ordered_case_by_prep)
    
    figsize=(12,12)
    if not prep == 'all':
        figsize=(12,12)
        
    f, ax = plt.subplots(figsize=figsize)
    case_by_prep = ordered_case_by_prep.plot(ax=ax, kind='bar', stacked=True)
    plt.ylabel('Count')
    plt.title(f'Number of cases for preposition: {prep}')
    ### возможно добавить подписи?
    
    print(case_by_prep)
    plt.savefig(f'graphics/boxplot_caseBy_prep={prep}.png')    

In [ ]:
case_by_prep(['cum'])

### Примеры для склоняемых частей речи после предлогов

In [ ]:
def get_info(count=10,
             pos=df_x['pos'].to_list(),
             case=['nominative', 'genitive', 'dative', 'accusative', 'locative', 'vocative'], 
             prep=['a', 'ab', 'de', 'cum', 'ex', 'e', 'sine', 'pro', 'prae']):
    
            
    df_by_case = df_x[(df_x['case'].isin(case)) & (df_x['prep'].isin(prep)) & (df_x['pos'].isin(pos))][['prep', 'word', 'pos', 'case', 'number', 'sentence']]
    sentence, prep, word, case_, num_, pos_ =   (df_by_case['sentence'].to_list(), df_by_case['prep'].to_list(), 
                                                 df_by_case['word'].to_list(), df_by_case['case'].to_list(),
                                                 df_by_case['number'].to_list(), df_by_case['pos'].to_list())

    
    a = list(zip(sentence, prep, word, case_, num_, pos_))
    examples = [random.choice(a) for i in range(count)]
    
    return len(a), examples

In [150]:
get_info(count=10, pos=['pronoun'])[1]

[('cur igitur mundus non animans sapiensque iudicetur, cum ex se procreet animantis atque sapientis?"',
  'ex',
  'se',
  'accusative',
  'singular',
  'pronoun'),
 ('id quoque a me impetrat.', 'a', 'me', 'accusative', 'singular', 'pronoun'),
 ('sunt aliae naturales quibusdam praeter- que vernam , quae suis constant sideribus—quorum ratio aptius reddetur tertio ab hoc volumine—, hiberna  aquilae exortu, aestiva canis ortu, tertia arcturi.',
  'ab',
  'hoc',
  'accusative',
  'singular',
  'pronoun'),
 ('Non curo istunc, de illa quaero.',
  'de',
  'illa',
  'accusative',
  'plural',
  'pronoun'),
 ('nam multo propius accedere ad scriptoris voluntatem eum, qui ex ipsius eam litteris interpretetur, quam illum, qui sententiam scriptoris non ex ipsius scripto spectet, quod ille suae voluntatis quasi imaginem reliquerit, sed domesticis suspicionibus perscrutetur.',
  'ex',
  'ipsius',
  'genitive',
  'singular',
  'pronoun'),
 ('dissuasimus nos, sed nihil de me, de Scipione dicam libentius.

### Предлог cum

![boxplot_caseBy_prep=['cum'].png](graphics/boxplot_caseBy_prep=['cum'].png)

In [ ]:
def count_by_undeclinable_lemma(pos, count_by='lemma', most_common=7):
    # count_by для совместимости с пунктуацией, у которой одна лемма...
    
    df = df_x[df_x['pos'] == pos]
        
    lemma_general = df.groupby(count_by, as_index=False).size()
    lemma_general_sorted = lemma_general.sort_values(ascending=False).reset_index()
    print(lemma_general_sorted)
    
    f, ax = plt.subplots(figsize=(18, 14))
    lemma_plot = sns.barplot(x=count_by, y=0, data=lemma_general_sorted.loc[0:most_common])
    plt.xticks(rotation=25)
    plt.ylabel('Count')
    plt.title(f'Count of entries by lemma for pos: {pos}')
    
    for bar in lemma_plot.patches:
            height = bar.get_height()
            if not np.isnan(height):
                height = int(height)
            if height == 0:
                continue

            ax.annotate(f'{height}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points", ha='center', va='bottom')

    print(lemma_plot)
    plt.savefig(f'graphics/boxplot_lemmaCount_pos={pos}.png')

In [ ]:
has_lemma = ['numeral', 'conjunction', 'exclamation', 'preposition', 'adverb']
no_lemma = ['punctuation']

In [ ]:
for pos in has_lemma:
    count_by_undeclinable_lemma(pos, most_common=7)

**NB** ex eo. Отфильтровать

In [ ]:
count_by_undeclinable_lemma(no_lemma[0], count_by='word', most_common=7)

In [ ]:
def get_info_und(count,
             pos=df_x['pos'].to_list(),
             prep=['a', 'ab', 'de', 'cum', 'ex', 'e', 'sine', 'pro', 'prae']):
        
    df_by_case = df_x[(df_x['prep'].isin(prep)) & (df_x['pos'].isin(pos))][['prep', 'word', 'pos', 'sentence']]
    sentence, prep, word, pos_ =   (df_by_case['sentence'].to_list(), df_by_case['prep'].to_list(), 
                                                 df_by_case['word'].to_list(), df_by_case['pos'].to_list())
    
    a = list(zip(prep, word, pos_, sentence))
    examples = [random.choice(a) for i in range(count)]
    
    return len(a), examples

In [148]:
get_info_und(10, pos=['punctuation'])[1]

[('ex',
  '-',
  'punctuation',
  'tremulis, spasticis, ex- ilientibus et quibus cor palpitet aliquid ex corde coctum mandendum ita, ut reliquae partis cinis cum cerebro hyaenae inlinatur;'),
 ('e', '.', 'punctuation', 'S. v. v. b. e. e. q. v. Cum pr.'),
 ('prae',
  '-',
  'punctuation',
  'lineas ex argento nigras prae- duci plerique mirantur .'),
 ('a',
  '.',
  'punctuation',
  'scripta epistula litterae mihi ante lucem a Lepta  Capua redditae sunt Idib. Mart. Pompeium a  Brundisio conscendisse, at Caesarem a. d. vii Kal.  Aprilis   Capuae fore.'),
 ('prae',
  '-',
  'punctuation',
  'Arabicae excellunt candore, circulo prae- lucido atque non gracili neque in recessu gemmae aut in deiectu renidente , sed in ipsis umbonibus nitente, prae-     terea substrato nigerrimi coloris .'),
 ('de',
  '-',
  'punctuation',
  'sic et morbo regio et hydropicis prodesse, etiam in choleris de- stillationes stomachi inhiberi.'),
 ('e',
  ',',
  'punctuation',
  'fica vi appellatum ter die fieri amar